In [ ]:
!pip install langchain

In [6]:
!pip show langchain

Name: langchain
Version: 0.1.6
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!pip install openai

In [9]:
!pip show openai

Name: openai
Version: 1.12.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llmx


In [ ]:
!pip install chromadb
!pip install tiktoken

In [ ]:
!pip install "unstructured[pdf]"

In [15]:
!pip install unstructured

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
import os
os.environ["OPENAI_API_KEY"]="sk-"
embedding = OpenAIEmbeddings()

In [13]:
!rm -rf ./docs/chroma/simple

In [14]:
from langchain_community.vectorstores.chroma import Chroma

J'importe d'une maniére globale mes documents **pdf** pour faire la jointure de l'ensemble de mes **pdf** se trouvant dans le dossier

```
 Data_PATH = "/content/sample_data"
```



In [17]:
from langchain.document_loaders import DirectoryLoader

Data_PATH = "/content/sample_data"

def load_document():
    loader = DirectoryLoader(Data_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

documents=load_document()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=2000,
    separators=["\n","\n\n","(?<=\.!-/)",""," "]

)
chunks=text_splitter.split_documents(documents)

Nombre de chunk

In [21]:
len(chunks)

5948

In [23]:
chunks[4].page_content

'2023 DSP 70 - Refonte du dispositif "Paris Méd\'" : vers un élargissement et un renforcement de l’aide à l’installation et au maintien des professionnel.le.s de santé libérales.aux. ........................................................................... 51\n\n2023 DSP 80 - Subvention (58.728 euros) et convention avec la "SELAS IPSO" pour la création d’un cabinet médical de groupe au 153, avenue d’Italie (13e) dans le cadre du dispositif "Paris Med’". .............................. 51\n\n2023 DSP 81 - Subvention (12.070 euros) et convention avec la "SELAS IPSO" pour l’agrandissement d’un cabinet médical de groupe au 73, rue de Montreuil (11e) dans le cadre du dispositif "Paris Med’". .................... 51\n\n2023 DSP 83 - Convention de mise à disposition d’équipements de santé dans le cadre de l’appel à projets pour une offre de soins de premier recours en soirée, week-ends et jours fériés (Paris centre, 15e). ..................... 51\n\nVœu déposé par le groupe Communiste et Cit

Créatiaon d'une database avec **chroma**

In [24]:
import chromadb.config
from langchain.vectorstores import Chroma

Je supprime l'environnement **chroma** si présent .

In [25]:
!rm -rf ./docs/chroma/simple

In [26]:
CHROMA_PATH='docs/chroma/simple'

In [ ]:
#if os.path.exists(CHROMA_PATH):
#  shutil.rmtree(CHROMA_PATH)

In [27]:
dbase=Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=CHROMA_PATH
)

Maintenant je crée le pipeline et choisir le llm text

**LLM et Pipeline**

In [28]:
from langchain import llms
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

J'utilise le **GPT-4** dans cette partie.

In [59]:
llm = llms.OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
chat_llm = ChatOpenAI(streaming=True,model="gpt-4",  callbacks=[StreamingStdOutCallbackHandler()], temperature=0,max_tokens=2500)

In [79]:
from langchain.prompts import PromptTemplate
template = """
Utilisez les éléments de contexte suivants pour répondre à la question à la fin.
Si vous ne connaissez pas la réponse, veuillez simplement indiquer que vous ne savez pas. Vous êtes un agent des services municipaux de Paris, veuillez vous en tenir aux informations officielles.
CONTEXTE : {context}
-------
EXTRAIT DU BULLETIN OFFICIEL DE DÉLIBÉRATIONS DE PARIS :
{chat_history}
Humain : {question}
Assistant :

"""

In [53]:
prompts=PromptTemplate(input_variables=["context", "question", "chat_history"], template=template)

In [54]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
# Prompt template
from langchain.prompts import PromptTemplate
# Conversational chain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# Streaming
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [80]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")
model_simple= ConversationalRetrievalChain.from_llm(chat_llm,
                                           dbase.as_retriever(),
                                           return_source_documents=True, memory=memory,
                                              combine_docs_chain_kwargs={"prompt": prompts})

In [56]:
model_simple("Quelles sont les subventions de la DEVE en 2023 ?")

En 2023, la Direction de l'Environnement et de l'Ecologie (DEVE) a attribué les subventions suivantes :

- DEVE 32 : Subventions de fonctionnement de 12.000 euros à deux associations œuvrant pour le développement de projets d’agriculture de proximité.
- DEVE 33 : Subvention d’équipement de 200.000 euros à la Fondation Terre de Liens et une subvention de fonctionnement de 37.000 euros à l'association "Terre de Liens IDF".
- DEVE 64 : Subvention de 25.000 euros à l’association "Un Plus Bio" pour l'accompagnement de la restauration collective parisienne.
- DEVE 66 : Subventions aux gestionnaires de la restauration collective parisienne dans le cadre de l’appel à manifestation d’intérêt de lutte contre le gaspillage alimentaire pour un montant total de 137.691.112 euros.
- DEVE 51 : Subvention de 100.000 euros à la Fondation de l’Armée du Salut pour son projet d’approvisionnement en alimentation durable et locale à destination des associations d’aide alimentaire Paris.
- DEVE 55 : Conventi

{'question': 'Quelles sont les subventions de la DEVE en 2023 ?',
 'chat_history': [HumanMessage(content='Quelles sont les subventions de la DEVE en 2023 ?'),
  AIMessage(content='En 2023, la Direction de l\'Environnement et de l\'Ecologie (DEVE) a attribué les subventions suivantes :\n\n- DEVE 32 : Subventions de fonctionnement de 12.000 euros à deux associations œuvrant pour le développement de projets d’agriculture de proximité.\n- DEVE 33 : Subvention d’équipement de 200.000 euros à la Fondation Terre de Liens et une subvention de fonctionnement de 37.000 euros à l\'association "Terre de Liens IDF".\n- DEVE 64 : Subvention de 25.000 euros à l’association "Un Plus Bio" pour l\'accompagnement de la restauration collective parisienne.\n- DEVE 66 : Subventions aux gestionnaires de la restauration collective parisienne dans le cadre de l’appel à manifestation d’intérêt de lutte contre le gaspillage alimentaire pour un montant total de 137.691.112 euros.\n- DEVE 51 : Subvention de 100.00

In [81]:
model_simple("Qu'est-ce que la DVD ?")

La DVD est l'acronyme de la Direction de la Voirie et des Déplacements. C'est un service de la Ville de Paris qui est en charge de la gestion et de l'aménagement des espaces publics, de la circulation et des transports.

{'question': "Qu'est-ce que la DVD ?",
 'chat_history': [HumanMessage(content="Qu'est-ce que la DVD ?"),
  AIMessage(content="La DVD est l'acronyme de la Direction de la Voirie et des Déplacements. C'est un service de la Ville de Paris qui est en charge de la gestion et de l'aménagement des espaces publics, de la circulation et des transports.")],
 'answer': "La DVD est l'acronyme de la Direction de la Voirie et des Déplacements. C'est un service de la Ville de Paris qui est en charge de la gestion et de l'aménagement des espaces publics, de la circulation et des transports.",
 'source_documents': [Document(page_content='Je mets aux voix, à main levée, le projet de délibération DVD 53.\n\nQui est pour ?\n\nQui est contre ?\n\nQui s\'abstient ?\n\nLe projet de délibération est adopté. (2023, DVD 53).\n\n2023 DVD 65 - Élaboration du Plan local de mobilité de Paris.\n\nVœu déposé par le groupe "Indépendants et Progressistes" relatif aux espaces de\n\nstationnement réservés à la location d